In [1]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.
C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# load ascii text and covert to lowercase
filename = "pride_and_prejudice.txt"
raw_text = open(filename).read()
raw_text = raw_text + open("hamlet.txt").read()
#raw_text = raw_text.lower()

In [3]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
#n_vocab = len(chars)
n_vocab = 86
print ("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print("Characters to int:\n",char_to_int)

Total Characters:  911214
Total Vocab:  86
Characters to int:
 {'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '?': 28, '@': 29, 'A': 30, 'B': 31, 'C': 32, 'D': 33, 'E': 34, 'F': 35, 'G': 36, 'H': 37, 'I': 38, 'J': 39, 'K': 40, 'L': 41, 'M': 42, 'N': 43, 'O': 44, 'P': 45, 'Q': 46, 'R': 47, 'S': 48, 'T': 49, 'U': 50, 'V': 51, 'W': 52, 'X': 53, 'Y': 54, 'Z': 55, '[': 56, ']': 57, '_': 58, 'a': 59, 'b': 60, 'c': 61, 'd': 62, 'e': 63, 'f': 64, 'g': 65, 'h': 66, 'i': 67, 'j': 68, 'k': 69, 'l': 70, 'm': 71, 'n': 72, 'o': 73, 'p': 74, 'q': 75, 'r': 76, 's': 77, 't': 78, 'u': 79, 'v': 80, 'w': 81, 'x': 82, 'y': 83, 'z': 84, '|': 85}


In [4]:
## prepare the dataset of input to output pairs encoded as integers
#seq_length = 30
#dataX = []
#dataY = []
#for i in range(0, n_chars - seq_length, 1):
#    seq_in = raw_text[i:i + seq_length]
#    seq_out = raw_text[i + seq_length]
#    dataX.append([char_to_int[char] for char in seq_in])
#    dataY.append(char_to_int[seq_out])

In [5]:
#n_patterns = len(dataX)
#print("Total Patterns: ", n_patterns)
## reshape X to be [samples, time steps, features]
#X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
## normalize
#X = X / float(n_vocab)
## one hot encode the output variable
#y = np_utils.to_categorical(dataY)

In [6]:
# define the LSTM model
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(30, 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(256))
model.add(Dropout(0.2))
model.add(Dense(86, activation='softmax'))

In [7]:
# load the network weights
filename = "weights-improvement-capital-cheat-20-1.5695.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
## pick a random seed
#start = numpy.random.randint(0, len(dataX)-1)
#pattern = dataX[start]
#print("Seed:")
#print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

In [9]:
## generate characters
#for i in range(1000):
#    x = numpy.reshape(pattern, (1, len(pattern), 1))
#    x = x / float(n_vocab)
#    prediction = model.predict(x, verbose=0)
#    index = numpy.argmax(prediction)
#    result = int_to_char[index]
#    seq_in = [int_to_char[value] for value in pattern]
#    sys.stdout.write(result)
#    pattern.append(index)
#    pattern = pattern[1:len(pattern)]
#print("\nDone.")

In [10]:
A = "e are two dogs in the red hous".lower()
A_num = []
for search_letter in A:
    A_num.append(char_to_int[search_letter])
    
print(A_num)

[63, 1, 59, 76, 63, 1, 78, 81, 73, 1, 62, 73, 65, 77, 1, 67, 72, 1, 78, 66, 63, 1, 76, 63, 62, 1, 66, 73, 79, 77]


In [11]:
import operator

x = numpy.reshape(A_num, (1, len(A_num), 1))
x = x / float(n_vocab)
prediction = model.predict(x, verbose=0)
prediction_dict = {}

for i in range(len(prediction[0])):
    prediction_dict[i] = prediction[0][i]
    
prediction_dict = dict(sorted(prediction_dict.items(), key=operator.itemgetter(1), reverse = True))

for i in range(5):
    print(i,")")
    print(list(prediction_dict.keys())[i])
    print("Character predicted: \t",int_to_char[list(prediction_dict.keys())[i]])
    print("Probability: \t\t",list(prediction_dict.values())[i])
    print("Resulting phrase:\t", A + int_to_char[list(prediction_dict.keys())[i]])

0 )
63
Character predicted: 	 e
Probability: 		 0.30949843
Resulting phrase:	 e are two dogs in the red house
1 )
1
Character predicted: 	  
Probability: 		 0.23894297
Resulting phrase:	 e are two dogs in the red hous 
2 )
12
Character predicted: 	 ,
Probability: 		 0.1082136
Resulting phrase:	 e are two dogs in the red hous,
3 )
14
Character predicted: 	 .
Probability: 		 0.08935467
Resulting phrase:	 e are two dogs in the red hous.
4 )
0
Character predicted: 	 

Probability: 		 0.045898724
Resulting phrase:	 e are two dogs in the red hous

